In [1]:
// TypeScript Jupyter extension
import * as tslab from "tslab";
import { requireCytoscape, requireCarbon } from "./lib/draw";

requireCytoscape();
requireCarbon();

# Sum Types

## Where Were We?

1. **Language primitives** (i.e., building blocks of languages)
    * Last time: closures and first-class functions
    * This time: **sum types**
2. Language paradigms (i.e., combinations of language primitives)
3. Building a language (i.e., designing your own language)

## Goal

1. Introduce the concept of **sum types**.
2. Highlight how we can use sum types along with the **type-system** to help us with **case analysis**.
3. Look at the tradeoffs between **compile-time** and **run-time** checking.

## Sum Types

We'll introduce sum types via example.

### Example 1

TypeScript division has "interesting" behavior.

In [2]:
1 / 2;  // Question: 0 or 0.5?

0.5


In [3]:
1 / 0;  // Question: error or NaN or infinity?

Infinity


Suppose we want to write a new divide function but make it so that division by 0 introduces an `undefined` value

In [4]:
function mySensibleDivideNoReturnType(a: number, b: number) {  // No type annotation
    if (b === 0) {
        return undefined;
    } else {
        return a / b;
    }
}

In [5]:
mySensibleDivideNoReturnType(1, 2);

0.5


In [6]:
mySensibleDivideNoReturnType(1, 0); // Undefined

In [7]:
type numberOrUndefined =  // Type declaration
  number      // number type
| undefined;  // "or" (|) undefined type

In [8]:
function mySensibleDivide(a: number, b: number): numberOrUndefined { // Uses sum type now
    if (b === 0) {
        const x: undefined = undefined;  // undefined case
        return x;
    } else {
        const x: number = a / b;  // number case
        return x;
    }
}

In [9]:
mySensibleDivide(1, 2);

0.5


In [10]:
mySensibleDivide(1, 0);

### Example 2

Now let's say I want to do arithmetic on `numberOrUndefined`.

In [11]:
// TypeScript numbers ...
const x = 0.5;
const y = undefined;
x + y

NaN


In [12]:
// TypeScript numbers ...
const x: number = 0.5;
const y: undefined = undefined;
x + y

4:1 - Operator '+' cannot be applied to types 'number' and 'undefined'.


In [13]:
// TypeScript numbers ...
mySensibleDivide(1, 2) + mySensibleDivide(1, 0) 

NaN


We can use the type system to help us out.

In [14]:
function incorrectMySensibleAdd(a: numberOrUndefined, b: numberOrUndefined): numberOrUndefined {
    if (typeof a === "number" && typeof b === "number") {
        return a + b;
    }
    else if (typeof a === "number" && typeof b === "undefined") {
        return a + b;  // Compiler throws and error here
    } else if (typeof a === "undefined" && typeof b === "undefined") {
        return undefined;
    }
}

6:16 - Operator '+' cannot be applied to types 'number' and 'undefined'.


In [ ]:
function mySensibleAdd(a: numberOrUndefined, b: numberOrUndefined): numberOrUndefined {
    if (typeof a === "number" && typeof b === "number") {
        return a + b;
    } else if (typeof a === "number" && typeof b === "undefined") {
        return undefined;
    } else if (typeof a === "undefined" && typeof b === "number") { 
        return undefined;
    } else if (typeof a === "undefined" && typeof b === "undefined") {
        return undefined;
    } else {
        throw Error("Shouldn't happen");  // Question: why shouldn't this happen?
    }
}

In [15]:
mySensibleAdd(mySensibleDivide(1, 2), mySensibleDivide(1, 0))

1:1 - Cannot find name 'mySensibleAdd'.


### Example 3

In [16]:
// TypeScript number and string behavior ...
console.log(1 + "a");
console.log(1 + 2 + "a");
console.log(1 + (2 + "a"));

1a
3a
12a


Suppose we want the behavior to always be string concatenation, lest we run into confusion about the difference between `1 + 2 + "a"` and `1 + (2 + "a")`.

In [17]:
type numberOrString = // Type declaration
  number   // number Type
| string;  // "or" (|) string type

In [18]:
function addNumberOrStringAttempt1(x1: numberOrString, x2: numberOrString): numberOrString {
    if (typeof x1 === "number" && typeof x2 === "number") {
        // Notice that we only run into trouble when we are adding numbers
        return "" + x1 + "" + x2;
    } else {
        return x1 + x2;
    }
}

6:16 - Operator '+' cannot be applied to types 'numberOrString' and 'numberOrString'.


### What's the problem?

- The TypeScript compiler is giving us an error at compile time because `numberOrString` does not have a `+` operator defined on that type.
- However, this might seem strange because we could add numbers with strings before.
- Let's get rid of the types and demonstrate that this is indeed the case.

In [19]:
function addNumberOrStringAttempt2(x1, x2): numberOrString {  // Notice that we got rid of the types
    if (typeof x1 === "number" && typeof x2 === "number") {
        // Notice that we only run into trouble when we are adding numbers
        return "" + x1 + "" + x2;
    } else {
        return x1 + x2;
    }
}

In [20]:
console.log(addNumberOrStringAttempt2(1, "a"));
console.log(addNumberOrStringAttempt2(addNumberOrStringAttempt2(1, 2), "a"));  // Notice that this
console.log(addNumberOrStringAttempt2(1, addNumberOrStringAttempt2(2, "a")));

1a
12a
12a


### What's the version with types?

- If we want to use types, we have to exhaustively list out all the cases.
- We can accomplish this by switching on each and every case.

In [21]:
function addNumberOrString(x1: numberOrString, x2: numberOrString): numberOrString {
    // Notice that we have to exhaustively list all 4 cases
    if (typeof x1 === "number" && typeof x2 === "number") {
        return "" + x1 + "" + x2;
    } else if (typeof x1 === "number" && typeof x2 === "string") {
        return x1 + x2; // Question: what is going on here?
    } else if (typeof x1 === "string" && typeof x2 === "number") {
        return x1 + x2;
    } else if (typeof x1 === "string" && typeof x2 === "string") {
        return x1 + x2;
    } else {
        // There are only 4 cases
        throw Error("Shouldn't happen.");
    }
}

In [22]:
console.log(addNumberOrString(1, "a"));
console.log(addNumberOrString(addNumberOrString(1, 2), "a"));  // Notice that this
console.log(addNumberOrString(1, addNumberOrString(2, "a")));

1a
12a
12a


## Digression: Literal Types

TypeScript provides **literal types** which enable language constants such as numbers and strings to be used as types.

### String literal type

In [23]:
type red = "red";  // The type red is the string "red"

In [24]:
const x: red = "red";  // Qeustion: What's the sum type of red?

In [25]:
// Compiler throws an error because the string "blue" is not the string "red"
const y: red = "blue";

2:7 - Type '"blue"' is not assignable to type '"red"'.


### Number literal type

In [26]:
type zero = 0;

In [27]:
const x: zero = 0;

In [28]:
// Type-error!
const y: zero = 1;

2:7 - Type '1' is not assignable to type '0'.


## Literal Types and Sum Types

Literal types by themselves may not seem that useful. They have many uses combined with sum types.

### Example 1: Many cases

In [29]:
type colors = 'red' | 'green' | 'blue';

function cycleColors(cs: colors): colors {
    if (cs === 'red') {
        return 'green';
    } else if (cs === 'green') {
        return 'blue';
    } else if (cs === 'blue') {
        return 'red';
    }
}

In [30]:
// I can't call cycleColors with an invalid string now
cycleColors('foobar');

2:13 - Argument of type '"foobar"' is not assignable to parameter of type 'colors'.


In [ ]:
// Typo is caught statically now
cycleColors('gren');

In [31]:
// Ensures our code is more safe
function codeBlock() {
    let color: colors = 'red';
    function callback(elmnt) {
        elmnt.style.color = color;
        color = cycleColors(color);
    }
    return callback;
}

function displayHTMLWithCallback(closure) {
    tslab.display.html(`
    <p onclick="callback(this)">Click me to change my text color.</p>

    <script>
    // We won't understand this fully for now, but we are essentially using code to do the copy-paste for us.
    ${closure.toString()}
    // Calling our closure will unpackage the function.
    const callback = ${closure.name}();
    </script>
    `);    
}

displayHTMLWithCallback(codeBlock)

Click me to change my text color.

### Example 2: Many cases 2

In [32]:
type HTTPClientErrorResponses =
  'BAD_REQUEST'
| 'UNAUTHORIZED'
| 'PAYMENT_REQUIRED'
| 'FORBIDDEN'
| 'NOT_FOUND'

function httpClientErrorResponsesToCode(c: HTTPClientErrorResponses): number {
    if (c === 'BAD_REQUEST') {
        return 400;
    } else if (c === 'UNAUTHORIZED') {
        return 401;
    } else if (c === 'PAYMENT_REQUIRED') {
        return 402;
    } else if (c === 'FORBIDDEN') {
        return 403;
    } else if (c === 'NOT_FOUND') {
        return 404;
    }
}

In [33]:
httpClientErrorResponsesToCode('BAD_REQUEST');

400


In [34]:
// typo is caught statically!
httpClientErrorResponsesToCode('BAD_REQUES');

2:32 - Argument of type '"BAD_REQUES"' is not assignable to parameter of type 'HTTPClientErrorResponses'.


### Example 3: Enforcing constraints at runtime versus compile-time

- Suppose we want to guarantee that an array of integers only contains the digits 0-9.
- That way we can convert an array of numbers into a string.

#### Approach 1: dynamic check

In [35]:
function digitsToString(ds: number[]): string {
    let tmp = "";
    for (const x of ds) {
        tmp += x;
    }
    return tmp;
}

In [36]:
digitsToString([-2, -1, 10]); // whoops

-2-110


In [37]:
function checkDigits(digits: number[]): boolean {
    for (const x of digits) {
        if (!Number.isInteger(x) || (x < 0 || x >= 10)) {
            return false;
        }
    }
    return true;
}

In [38]:
console.log(checkDigits([1, 2, 3]));
console.log(checkDigits([-1, 2]));

true
false


In [39]:
function digitsToString2(ds: number[]): string | false { // string or false
    return checkDigits(ds) ? digitsToString(ds) : false;
}

In [40]:
console.log(digitsToString2([1, 2, 3]));
console.log(digitsToString2([-1, 2]));

123
false


#### Or we could try to encode this directly in the type system.

In [41]:
type digit = 0|1|2|3|4|5|6|7|8|9
type digits = digit[]

In [42]:
const x: digits = [3, 5, 1, 2];

In [43]:
// Type-error!
const y: digits = [3, 5, 1, 2, -1];

2:32 - Type '-1' is not assignable to type 'digit'.


In [ ]:
function digitsToString3(ds: digits): string {  // instead of number[]
    return digitsToString(ds);
}

In [44]:
digitsToString3(x);

1:1 - Cannot find name 'digitsToString3'. Did you mean 'digitsToString2'?


In [ ]:
// Type-error!
// digitsToString3([3, 5, 1, 2, -1]);
digitsToString3([3, 5, 1, 2, 0, 9, 'hi']);

## Example 4: Tradeoffs, tradeoffs

There's a tradeoff between trying to enforce constraints statically (i.e., at compile time) vs. at run-time.
We'll see this in the makings of a Chipotle burrito.

### Making a Chipotle burrito

In [45]:
type Rice = "WHITE" | "BROWN";
type Beans = "BLACK" | "PINTO";
type Meat = "CHICKEN" | "CARNITAS" | "STEAK" | "SOFRITAS";
type Salsa = "MILD" | "MEDIUM" | "HOT";

type Burrito = {
    rice: Rice,
    beans: Beans,
    meat: Meat,
    salsa: Salsa
};

In [46]:
function makeBurrito(r: Rice, b: Beans, m: Meat, s: Salsa): Burrito {
    return {
        rice: r,
        beans: b,
        meat: m,
        salsa: s,
    };
}

In [47]:
makeBurrito("WHIT", "PINTO", "CHICKEN", "HOT");

1:13 - Argument of type '"WHIT"' is not assignable to parameter of type 'Rice'.


### Abstracting Chipotle burritos. Burrito's will make another guest appearance later on ...

In [48]:
// A container that contains 4 items
type Container4<S, T, U, V> = {
    item1: S,
    item2: T,
    item3: U,
    item4: V
};

In [49]:
// A function that creates a contianer that holds 4 items
function makeContainer4<S, T, U, V>(s: S, t: T, u: U, v: V): Container4<S, T, U, V> {
    return {
        item1: s,
        item2: t,
        item3: u,
        item4: v
    };
}

In [50]:
// Sometimes we would like a container that only holds 3 items
// 
type Container3<S, T, U> = {
    item1: S,
    item2: T,
    item3: U,
};

In [51]:
function dropOneItem<S, T, U, V>(
    position: 1|2|3|4,
    container4: Container4<S, T, U, V>): Container3<T, U, V> // 4 cases after we drop 1 item
                                       | Container3<S, U, V>
                                       | Container3<S, T, V>
                                       | Container3<S, T, U>
{
    if (position === 1) {
        return { 
            item1: container4.item2,
            item2: container4.item3,
            item3: container4.item4
        } as unknown as Container3<T, U, V>;
    } else if (position === 2) {
        return {
            item1: container4.item1,
            item2: container4.item3,
            item3: container4.item4
        } as unknown as Container3<S, U, V>;
    } else if (position === 3) {
        return {
            item1: container4.item1,
            item2: container4.item2,
            item3: container4.item4
        } as unknown as Container3<S, T, V>;
    } else {
        return {
            item1: container4.item1,
            item2: container4.item2,
            item3: container4.item3
        } as unknown as Container3<S, T, U>;
    }
}

In [52]:
// A burrito uses a tortilla as a container
type TortillaContainer = Container4<Rice, Beans, Meat, Salsa>;

In [53]:
dropOneItem(2, makeContainer4("BROWN", "BLACK", "STEAK", "MILD"))

{ item1: 'BROWN', item2: 'STEAK', item3: 'MILD' }


In [54]:
// Type error!
dropOneItem(5, makeContainer4("BROWN", "BLACK", "STEAK", "MILD"))

2:13 - Argument of type '5' is not assignable to parameter of type '1 | 2 | 3 | 4'.


## Summary

- Sum types enable us to express in the type system different cases.
- Literal types enable us to use numbers and strings as types.
- By combining the two, we can have a light-weight method to get the type-system to ensure that our code is correct at compile-time when performing case analysis.
- There's a tradeoff in encoding information in types.